### Merging all categories grouped on HADM_ID

In [1]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
import re
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences

/usr/local/lib/python3.5/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
# create output path
mypath_input = "/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/"
mypath_output = "/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/"
os.makedirs(mypath_output, exist_ok=True)

In [12]:
file_list = list(filter(lambda k: '.json' in k, os.listdir(mypath_input)))

In [13]:
file_list

['microbioevents.json',
 'noteevents.json',
 'labevents.json',
 'inputevents_mv.json',
 'all_events.json',
 'chartevents.json',
 'procedureevents.json',
 'inputevents_cv.json',
 'prescriptionevents.json',
 'outputevents.json']

In [14]:
#Remove chartevents and all_events fromt he files list
if 'chartevents.json' in file_list:
    file_list.remove('chartevents.json')
    print('Chartevents removed from list')
else :
    print('Chartevents not found in list')
if 'all_events.json' in file_list:
    file_list.remove('all_events.json')
    print('all_events removed from list')
else :
    print('all_events not found in list')
    

Chartevents removed from list
all_events removed from list


In [15]:
df = pd.read_json(mypath_input+"chartevents.json")

In [16]:
file_list

['microbioevents.json',
 'noteevents.json',
 'labevents.json',
 'inputevents_mv.json',
 'procedureevents.json',
 'inputevents_cv.json',
 'prescriptionevents.json',
 'outputevents.json']

In [17]:
for i in tqdm(file_list):
    x = re.sub('.json','',i)
    df1 = pd.read_json(mypath_input+i)
    df1 = df1.rename({'events':x}, axis = 'columns')
    df = df.merge(df1[['HADM_ID',x]], how='left',left_on = 'HADM_ID',right_on = 'HADM_ID')

In [18]:
column_list = df.columns.tolist()

In [19]:
column_list.remove('HADM_ID')
column_list.remove('HOSPITAL_EXPIRE_FLAG')
column_list.remove('SUBJECT_ID')
column_list

['events',
 'microbioevents',
 'noteevents',
 'labevents',
 'inputevents_mv',
 'procedureevents',
 'inputevents_cv',
 'prescriptionevents',
 'outputevents']

In [20]:
df.head()

,HADM_ID,HOSPITAL_EXPIRE_FLAG,SUBJECT_ID,events,microbioevents,noteevents,labevents,inputevents_mv,procedureevents,inputevents_cv,prescriptionevents,outputevents
0,163353,0,2,"[Resp, Rate, 3603-39-Breath, Resp, Rate, 3603-...","[BLOOD, CULTURE, -, NEONATE, Negative, Culture]","[Nursing/other, Report, Nursing, Transfer, not...","[Atypical, Lymphocytes, Blood, Hematology, 511...",NaN,NaN,NaN,"[MAIN, NEOIVGentamicin, NaN, DRUG_CD-GENT10I, ...",NaN
1,145834,0,3,"[Sputum, Source, 657-Suctioned, Riker, SAS, Sc...","[SWAB, PROBABLE, ENTEROCOCCUS, SWAB, YEAST, SW...","[Echo, Report, PATIENT/TEST, INFORMATION:, Ind...","[Anion, Gap, Blood, Chemistry, 50868-17-mEq/L,...",NaN,NaN,"[D5W, Intravenous, Push, 30013-250-ml, D5W, In...",NaN,"[Urine, Out, Foley, 40055-20-ml, Urine, Out, F..."
2,143045,0,13,"[Diagnosis/op, 917-CORONARY ARTERY DISEASE, Re...",NaN,"[ECG, Report, Sinus, rhythm/, Non-diagnostic, ...","[Alanine, Aminotransferase, (ALT), Blood, Chem...",NaN,NaN,"[Po, Intake, Oral, 30056-80-ml, Po, Intake, Or...","[BASE, D5W, NaN, DRUG_CD-HEPBASE, NaN, NaN, IV...","[Urine, Out, Foley, 40055-150-ml, Urine, Out, ..."
3,128744,0,105,"[Riker, SAS, Scale, 1337-Calm/Cooperative, RUL...","[URINE, PROBABLE, ENTEROCOCCUS]","[ECG, Report, Sinus, rhythm., No, change, sinc...","[Lactate, Blood, Blood, Gas, 50813-2.5-mmol/L,...",NaN,NaN,"[.9%, Normal, Saline, Intravenous, Push, 30018...","[BASE, NS, NaN, DRUG_CD-NS1000, GSN-001210, ND...","[Pre-Admission, Output, Pre-Admission, Output,..."
4,166988,0,887,"[GI, Intub, 1, Type, 200-Salem Sump, GI, Intub...","[BLOOD, CULTURE, Negative, Culture, BLOOD, CUL...","[ECG, Report, Technically, difficult, study, S...","[Acetaminophen, Blood, Chemistry, Alanine, Ami...",NaN,NaN,"[D5W, Intravenous, Push, 30013-250-ml, D5W, In...","[MAIN, Sodium, Chloride, 0.9%, Flush, Sodium, ...","[Urine, Out, Foley, 40055-150-ml, Urine, Out, ..."


In [31]:
df.fillna('NaN', inplace=True)

In [32]:
for i in tqdm(column_list):
    df[i] = df[i].fillna(0).map(lambda x: [x] if x == 'NaN' else x)

In [33]:
df.head()

,HADM_ID,HOSPITAL_EXPIRE_FLAG,SUBJECT_ID,events,microbioevents,noteevents,labevents,inputevents_mv,procedureevents,inputevents_cv,prescriptionevents,outputevents
0,163353,0,2,"[Resp, Rate, 3603-39-Breath, Resp, Rate, 3603-...","[BLOOD, CULTURE, -, NEONATE, Negative, Culture]","[Nursing/other, Report, Nursing, Transfer, not...","[Atypical, Lymphocytes, Blood, Hematology, 511...",[NaN],[NaN],[NaN],"[MAIN, NEOIVGentamicin, NaN, DRUG_CD-GENT10I, ...",[NaN]
1,145834,0,3,"[Sputum, Source, 657-Suctioned, Riker, SAS, Sc...","[SWAB, PROBABLE, ENTEROCOCCUS, SWAB, YEAST, SW...","[Echo, Report, PATIENT/TEST, INFORMATION:, Ind...","[Anion, Gap, Blood, Chemistry, 50868-17-mEq/L,...",[NaN],[NaN],"[D5W, Intravenous, Push, 30013-250-ml, D5W, In...",[NaN],"[Urine, Out, Foley, 40055-20-ml, Urine, Out, F..."
2,143045,0,13,"[Diagnosis/op, 917-CORONARY ARTERY DISEASE, Re...",[NaN],"[ECG, Report, Sinus, rhythm/, Non-diagnostic, ...","[Alanine, Aminotransferase, (ALT), Blood, Chem...",[NaN],[NaN],"[Po, Intake, Oral, 30056-80-ml, Po, Intake, Or...","[BASE, D5W, NaN, DRUG_CD-HEPBASE, NaN, NaN, IV...","[Urine, Out, Foley, 40055-150-ml, Urine, Out, ..."
3,128744,0,105,"[Riker, SAS, Scale, 1337-Calm/Cooperative, RUL...","[URINE, PROBABLE, ENTEROCOCCUS]","[ECG, Report, Sinus, rhythm., No, change, sinc...","[Lactate, Blood, Blood, Gas, 50813-2.5-mmol/L,...",[NaN],[NaN],"[.9%, Normal, Saline, Intravenous, Push, 30018...","[BASE, NS, NaN, DRUG_CD-NS1000, GSN-001210, ND...","[Pre-Admission, Output, Pre-Admission, Output,..."
4,166988,0,887,"[GI, Intub, 1, Type, 200-Salem Sump, GI, Intub...","[BLOOD, CULTURE, Negative, Culture, BLOOD, CUL...","[ECG, Report, Technically, difficult, study, S...","[Acetaminophen, Blood, Chemistry, Alanine, Ami...",[NaN],[NaN],"[D5W, Intravenous, Push, 30013-250-ml, D5W, In...","[MAIN, Sodium, Chloride, 0.9%, Flush, Sodium, ...","[Urine, Out, Foley, 40055-150-ml, Urine, Out, ..."


In [34]:
df.to_json(mypath_output+"all_events.json")